In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [8]:
df_accounts = spark.read \
    .format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load('./data/accounts.csv')

df_accounts.show()

+---+----------+---------+---+-------+
| id|first_name|last_name|age|country|
+---+----------+---------+---+-------+
|  1|     Darcy| Phillips| 24|     YE|
|  2|    Amelia|   Wright| 66|     CN|
|  3|     Haris|    Ellis| 61|     CR|
|  4|      Tony|     Hall| 51|     JO|
|  5|     Rubie|  Stewart| 27|     RO|
|  6|     Miley|    Perry| 27|     ZA|
|  7|    Marcus|   Carter| 66|     CN|
|  8|   Charlie|   Harris| 22|     SR|
|  9|     Honey|   Rogers| 60|     IL|
| 10|      Luke|   Harris| 66|     IR|
| 11|     Spike|   Murphy| 57|     IN|
| 12|   Vincent|    Adams| 51|     CN|
| 13|     James|   Barnes| 56|     HK|
| 14|    George|   Bailey| 18|     AT|
| 15|    Sienna|   Holmes| 48|     ZW|
| 16|  Isabella|  Elliott| 47|     CO|
| 17|   Freddie|   Martin| 47|     SC|
| 18|      Kate|   Wright| 51|     BE|
| 19|    Albert|    Myers| 24|     BM|
| 20|    Connie|    Wells| 64|     SC|
+---+----------+---------+---+-------+
only showing top 20 rows



In [10]:
df_accounts.count()

500000

In [9]:
df_country_abbreviation = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .load('./data/country_abbreviation-original.csv')

df_country_abbreviation.show()

+-----------------+------------+
|country_full_name|abbreviation|
+-----------------+------------+
|        Argentina|          AR|
|        Australia|          AU|
|          Austria|          AT|
|          Bahamas|          BS|
|          Bahrain|          BH|
|       Bangladesh|          BD|
|         Barbados|          BB|
|          Belgium|          BE|
|           Belize|          BZ|
|            Benin|          BJ|
|          Bermuda|          BM|
|          Bolivia|          BO|
|           Brazil|          BR|
|         Bulgaria|          BG|
|     Burkina Faso|          BF|
|            Chile|          CL|
|            China|          CN|
|         Colombia|          CO|
|       Costa Rica|          CR|
|   Côte D' Ivoire|          CI|
+-----------------+------------+
only showing top 20 rows



In [12]:
df_country_abbreviation.count()

121

In [10]:
df_transactions = spark.read \
    .format("csv") \
    .option("header","true") \
    .option("inferSchema","true").option("delimiter",";") \
    .load('./data/transactions-original.csv')

df_transactions.show()

+------+-------+------------+----------------+-------+
|    id| amount|account_type|transaction_date|country|
+------+-------+------------+----------------+-------+
|179528|-730.86|    Business|      2013-07-10|     SV|
|378343|-946.98|    Personal|      2018-04-06|     YE|
| 75450|7816.92|Professional|      2016-11-20|     SI|
|357719| 704.02|    Business|      2016-11-06|     ID|
|110511| 3462.6|    Personal|      2018-01-18|     BS|
|461830| 762.81|Professional|      2017-06-20|     CN|
| 30180|5390.24|Professional|      2021-05-26|     GN|
| 65398|4765.77|    Personal|      2018-05-01|     TR|
|170899|8775.89|    Business|      2013-10-16|     SK|
|234300|8455.18|Professional|      2015-10-06|     LU|
|208027| 6244.1|    Business|      2020-03-06|     AE|
|161212|5904.56|    Personal|      2016-09-07|     EG|
|105372|4079.76|Professional|      2015-02-12|     MT|
|205321| 3570.4|Professional|      2012-07-02|     MU|
|410863|2328.83|    Business|      2012-12-20|     SR|
|486752| 5

In [14]:
df_transactions.count()

5000000

In [12]:
df_transactions.createOrReplaceTempView("transactions")
df_country_abbreviation.createOrReplaceTempView("country_abbreviation")
df_accounts.createOrReplaceTempView("accounts")

In [24]:
# Calculate how many accounts of each type there are using Spark SQL. The return type is a
# dataframe [account_type: string, account_type_count: int]

aggr_df = spark.sql("""
    SELECT account_type, count(*) as account_type_count 
    FROM transactions 
    GROUP BY account_type
""")

aggr_df.show()

+------------+------------------+
|account_type|account_type_count|
+------------+------------------+
|    Personal|           1667072|
|Professional|           1667358|
|    Business|           1665570|
+------------+------------------+



In [23]:
# Calculate only the balance and the latest date for each account from transactions.csv. 
# To calculate the balance, summarize all the transactions for each account. The return type is
# a dataframe [account_id: string, balance: string, latest_date: date].

balance_df = spark.sql("""
    SELECT id, SUM(amount) 
    FROM transactions 
    GROUP BY id
""")

balance_df.show()


+------+------------------+
|    id|       sum(amount)|
+------+------------------+
|482333|          27174.07|
|222048|          48004.81|
|328078|          36948.25|
|192401|          36736.98|
|273916| 47475.37999999999|
|485103|          62198.93|
|300282|          55103.62|
| 20683|          56448.72|
| 15846|          58671.91|
|446783| 98085.51000000001|
| 92182|           42335.3|
|477485|          22114.03|
|171142|40428.899999999994|
|317762|          40025.55|
| 65478| 57941.90000000001|
|306768|          26566.93|
|380411|          43652.94|
|304681|          37827.69|
|475638|44509.100000000006|
| 97413|          39611.24|
+------+------------------+
only showing top 20 rows



In [27]:
ddf = spark.sql("""
    SELECT *
    FROM transactions 
    WHERE id = 482333
    ORDER BY transaction_date
""")

ddf.show()


+------+-------+------------+----------------+-------+
|    id| amount|account_type|transaction_date|country|
+------+-------+------------+----------------+-------+
|482333|7269.27|    Business|      2012-08-18|     ID|
|482333| 123.38|    Personal|      2013-09-01|     SE|
|482333|5856.55|    Business|      2015-11-17|     PT|
|482333|2049.52|    Personal|      2016-07-22|     LB|
|482333|-609.49|    Personal|      2016-11-10|     VN|
|482333|  -32.0|Professional|      2016-12-24|     HR|
|482333|3967.39|    Personal|      2019-03-03|     IQ|
|482333| 2049.5|    Personal|      2019-09-06|     BO|
|482333|6499.95|Professional|      2020-07-17|     ID|
+------+-------+------------+----------------+-------+



In [26]:
ddf2 = spark.sql("""
    SELECT MIN(transaction_date), MAX(transaction_date)
    FROM transactions 
    WHERE id = 482333
""")

ddf2.show()


+---------------------+---------------------+
|min(transaction_date)|max(transaction_date)|
+---------------------+---------------------+
|           2012-08-18|           2020-07-17|
+---------------------+---------------------+



In [29]:
ddf3 = spark.sql("""
    SELECT id, MIN(transaction_date), MAX(transaction_date)
    FROM transactions t
    GROUP BY id
""")

ddf3.show()



+------+---------------------+---------------------+
|    id|min(transaction_date)|max(transaction_date)|
+------+---------------------+---------------------+
|482333|           2012-08-18|           2020-07-17|
|222048|           2011-07-22|           2020-07-20|
|328078|           2011-10-09|           2020-02-01|
|192401|           2011-05-06|           2020-01-30|
|273916|           2012-04-09|           2021-05-30|
|485103|           2011-02-17|           2021-05-22|
|300282|           2012-04-28|           2021-05-01|
| 20683|           2011-05-16|           2021-10-27|
| 15846|           2013-01-18|           2020-12-23|
|446783|           2011-05-11|           2021-12-11|
| 92182|           2013-03-14|           2020-08-08|
|477485|           2011-05-23|           2020-05-23|
|171142|           2012-05-01|           2021-04-07|
|317762|           2011-08-22|           2021-12-02|
| 65478|           2011-02-14|           2021-10-06|
|306768|           2011-12-04|           2019-

In [36]:

balance_df = spark.sql("""
    WITH sum_balance AS (
        SELECT id, SUM(amount) as balance
        FROM transactions 
        GROUP BY id
    ), 
    maxd AS (
        SELECT id, MAX(transaction_date) as max_date
        FROM transactions t
        GROUP BY id
    )
    SELECT DISTINCT t.id, sb.balance, md.max_date
    FROM transactions t
    JOIN sum_balance sb ON t.id = sb.id
    JOIN maxd md ON md.id = sb.id
    
""")

balance_df.show()

+---+------------------+----------+
| id|           balance|  max_date|
+---+------------------+----------+
|  1| 51909.75000000001|2021-08-19|
|  2|58346.670000000006|2021-02-25|
|  3|          33709.88|2020-05-04|
|  4|30160.090000000004|2021-01-06|
|  5|          34593.07|2020-05-31|
|  6|          55611.34|2021-07-01|
|  7|45952.439999999995|2021-09-08|
|  8|          40101.59|2020-05-14|
|  9|          46625.15|2019-01-03|
| 10|          50807.19|2020-12-27|
| 11|          25039.38|2021-08-24|
| 12|32998.329999999994|2021-05-16|
| 13|          39822.72|2021-09-02|
| 14|          35763.46|2020-03-17|
| 15|39481.770000000004|2021-11-07|
| 16|          52134.41|2017-07-16|
| 17|23278.489999999998|2021-12-14|
| 18|26161.980000000003|2020-02-27|
| 19| 43046.28999999999|2021-05-27|
| 20|          41532.67|2020-11-15|
+---+------------------+----------+
only showing top 20 rows



In [37]:
ddf4 = spark.sql("""
    SELECT id, MIN(transaction_date), MAX(transaction_date)
    FROM transactions t
    WHERE id = 1 or id = 2
    GROUP BY id
""")

ddf4.show()


+---+---------------------+---------------------+
| id|min(transaction_date)|max(transaction_date)|
+---+---------------------+---------------------+
|  1|           2011-04-08|           2021-08-19|
|  2|           2012-01-30|           2021-02-25|
+---+---------------------+---------------------+



In [38]:
df5 = spark.sql("""
    SELECT id, SUM(amount) 
    FROM transactions 
    WHERE id = 1 or id = 2
    GROUP BY id
""")

df5.show()


+---+------------------+
| id|       sum(amount)|
+---+------------------+
|  1| 51909.75000000001|
|  2|58346.670000000006|
+---+------------------+



In [ ]:
# W34  21.08.23 
Spark API 
2. Write a function using Spark Python or Spark Scala API to calculate total earnings (sum of
transactions above 0) for each user from Switzerland by year as a pivot table. 

The result dataframe should contain:
- user full names as one field split by whitespace, years, and earning values.

Note: The real datasets could be more extensive than provided csv samples.

3. Store the AI chat conversation in the "Task_2.txt" file and commit it to the repository. If you
did without AI support, create an empty file.

4. Commit C#4 with the "UC#33 Task_2" message.


In [14]:
ddf6 = spark.sql("""
    SELECT *
    FROM country_abbreviation
    WHERE country_full_name = 'Switzerland'
""")

ddf6.show()

+-----------------+------------+
|country_full_name|abbreviation|
+-----------------+------------+
|      Switzerland|          CH|
+-----------------+------------+



In [16]:
ddf7 = spark.sql("""
    SELECT t.id, SUM(amount)
    FROM country_abbreviation ca
    JOIN transactions t ON t.country = ca.abbreviation
    WHERE country_full_name = 'Switzerland'
    GROUP BY id
    HAVING SUM(amount) > 0 
""")

ddf7.show()

 

+------+-----------+
|    id|sum(amount)|
+------+-----------+
|283980|    2612.92|
|  6357|    9826.82|
|226243|    9103.32|
| 44437|    2909.02|
| 81501|    2364.32|
|145504|    1989.41|
|251742|    8526.42|
|341687|    8949.24|
| 30903|    9668.23|
|453490|    7411.66|
|  7240|    1485.11|
| 89874|    6253.26|
| 47217|    8946.42|
|284346|    9731.56|
|420759|    8314.65|
|376414|     5606.8|
|474711|     8189.4|
|346791|    1983.52|
|188488|     979.04|
|454924|    2795.71|
+------+-----------+
only showing top 20 rows



In [19]:
ddf8 = spark.sql("""
    SELECT t.id, t.transaction_date, SUM(amount)
    FROM country_abbreviation ca
    JOIN transactions t ON t.country = ca.abbreviation
    WHERE country_full_name = 'Switzerland'
    GROUP BY id, transaction_date 
    ORDER BY id
""")

ddf8.show()

+---+----------------+-----------+
| id|transaction_date|sum(amount)|
+---+----------------+-----------+
|  8|      2015-09-29|    3481.48|
| 19|      2017-01-04|    5630.04|
| 20|      2020-03-16|    -432.68|
| 48|      2020-09-09|    7814.77|
| 51|      2020-12-21|    8011.46|
| 63|      2019-09-11|    -774.16|
| 66|      2021-07-18|    6961.53|
| 76|      2019-08-12|    8116.98|
| 76|      2020-03-20|      204.7|
| 77|      2020-08-05|     1298.8|
| 80|      2011-09-29|    1074.49|
| 86|      2013-11-12|      673.9|
| 98|      2012-07-20|    9278.56|
|101|      2012-09-19|    -882.35|
|104|      2021-07-29|     705.88|
|121|      2020-06-11|    1230.22|
|132|      2012-03-07|    4971.14|
|136|      2015-07-24|    8145.67|
|145|      2019-01-11|    1801.25|
|160|      2021-07-28|    5702.53|
+---+----------------+-----------+
only showing top 20 rows



In [20]:
ddf9 = spark.sql("""
    SELECT id,	first_name,	last_name
    FROM accounts
""")

ddf9.show()

+---+----------+---------+
| id|first_name|last_name|
+---+----------+---------+
|  1|     Darcy| Phillips|
|  2|    Amelia|   Wright|
|  3|     Haris|    Ellis|
|  4|      Tony|     Hall|
|  5|     Rubie|  Stewart|
|  6|     Miley|    Perry|
|  7|    Marcus|   Carter|
|  8|   Charlie|   Harris|
|  9|     Honey|   Rogers|
| 10|      Luke|   Harris|
| 11|     Spike|   Murphy|
| 12|   Vincent|    Adams|
| 13|     James|   Barnes|
| 14|    George|   Bailey|
| 15|    Sienna|   Holmes|
| 16|  Isabella|  Elliott|
| 17|   Freddie|   Martin|
| 18|      Kate|   Wright|
| 19|    Albert|    Myers|
| 20|    Connie|    Wells|
+---+----------+---------+
only showing top 20 rows



In [22]:
ddf10 = spark.sql("""
    SELECT * 
    FROM transactions t
    WHERE id = 8 
""")

ddf10.show()

# +---+-------+------------+----------------+-------+
# | id| amount|account_type|transaction_date|country|
# +---+-------+------------+----------------+-------+
# |  8|3481.48|Professional|      2015-09-29|     CH|
# +---+-------+------------+----------------+-------+

+---+-------+------------+----------------+-------+
| id| amount|account_type|transaction_date|country|
+---+-------+------------+----------------+-------+
|  8|7685.97|    Personal|      2020-05-14|     LU|
|  8|8150.49|Professional|      2012-01-20|     PK|
|  8|6095.47|Professional|      2019-12-27|     BO|
|  8|4904.49|    Business|      2012-07-15|     FR|
|  8|3511.23|Professional|      2012-05-24|     LR|
|  8|3481.48|Professional|      2015-09-29|     CH|
|  8|6272.46|    Personal|      2013-11-16|     BB|
+---+-------+------------+----------------+-------+



In [26]:

ddf11 = spark.sql("""
    SELECT id, transaction_date, country 
    FROM transactions t
    WHERE id IN ( 8,  19,  20,  48,  51,  63,  66,  76,  76,  77,  80,  86,  98, 101, 104, 121, 132, 136, 145, 160) 
        AND COUNTRY = 'CH'
    ORDER BY id
""")

ddf11.show()

+---+----------------+-------+
| id|transaction_date|country|
+---+----------------+-------+
|  8|      2015-09-29|     CH|
| 19|      2017-01-04|     CH|
| 20|      2020-03-16|     CH|
| 48|      2020-09-09|     CH|
| 51|      2020-12-21|     CH|
| 63|      2019-09-11|     CH|
| 66|      2021-07-18|     CH|
| 76|      2020-03-20|     CH|
| 76|      2019-08-12|     CH|
| 77|      2020-08-05|     CH|
| 80|      2011-09-29|     CH|
| 86|      2013-11-12|     CH|
| 98|      2012-07-20|     CH|
|101|      2012-09-19|     CH|
|104|      2021-07-29|     CH|
|121|      2020-06-11|     CH|
|132|      2012-03-07|     CH|
|136|      2015-07-24|     CH|
|145|      2019-01-11|     CH|
|160|      2021-07-28|     CH|
+---+----------------+-------+



In [29]:
ddf12 = spark.sql("""
    WITH us AS (
        SELECT a.id,	a.first_name,	a.last_name
        FROM accounts a
    ), trrr (
        SELECT t.id, t.transaction_date, SUM(amount) as AMT
        FROM country_abbreviation ca
        JOIN transactions t ON t.country = ca.abbreviation
        JOIN us u ON u.id = t.id
        WHERE country_full_name = 'Switzerland'
        GROUP BY t.id, t.transaction_date 
        HAVING SUM(amount) > 0
        ORDER BY t.id
    )    
    SELECT u.first_name, u.last_name, tr.transaction_date, tr.AMT
    FROM trrr tr 
    JOIN us u ON tr.id = u.id 
""")

ddf12.show()

+----------+---------+----------------+-------+
|first_name|last_name|transaction_date|    AMT|
+----------+---------+----------------+-------+
|    Miller|    Grant|      2017-12-20|8969.46|
|     Daryl| Richards|      2017-08-30|3218.19|
|      Maya|    Perry|      2011-05-15|6686.35|
|     Daisy|    Clark|      2014-08-09|3932.69|
|    Arthur|Armstrong|      2018-12-29|1842.69|
| Annabella|  Higgins|      2011-07-28|1171.24|
|    Wilson| Phillips|      2013-10-27| 1960.1|
|   Vincent|  Johnson|      2011-12-02| 9490.7|
|    Carlos|    Allen|      2018-11-26|9485.89|
|     Sofia| Johnston|      2013-01-06|9729.65|
|    Vivian|    Grant|      2018-07-18|5106.76|
|    Oliver| Ferguson|      2018-06-29|1106.76|
|      Adam|   Brooks|      2018-12-07|4130.63|
|    Arnold|   Harris|      2011-10-17|9708.46|
|  Isabella|    Jones|      2016-03-03|7111.15|
|    Kellan|    Smith|      2018-04-06|6269.93|
|   Richard|   Howard|      2018-08-25|5215.36|
|      Lana|    Craig|      2016-05-18|3

In [ ]:
ddf13  